In [32]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [35]:
df = pd.read_csv("stock_market_articles.csv")
df['pubDate'] = pd.to_datetime(df['pubDate'])
df

,title,pubDate
0,recession now... or stagflation forever,2024-09-13 16:45:00
1,"mortgage rates are dropping, but homes are not...",2024-09-13 16:40:00
2,hospitality stocks out-innovate challenges as ...,2024-09-13 16:34:47
3,ratings agency fitch says extended strike at b...,2024-09-13 16:30:33
4,"google parent company in bear territory, down ...",2024-09-13 16:28:47
...,...,...
4632,a bull market is here: 2 brilliant stocks down...,2024-11-30 10:00:00
4633,7 christmas movies you can watch with hulu's 9...,2024-11-30 10:00:00
4634,jefferies: 'are we near the rebirth of new nuk...,2024-11-30 10:00:00
4635,deutsche bank is out with its 2025 outlook for...,2024-11-30 10:00:00


In [36]:
def fetch_sp500_data(date):
    sp500 = yf.Ticker("^GSPC")  # S&P 500 ticker symbol
    # Fetch historical data around the given date
    history = sp500.history(
        start=(date - pd.Timedelta(days=7)).strftime('%Y-%m-%d'),
        end=(date + pd.Timedelta(days=7)).strftime('%Y-%m-%d')
    )
    return history

def get_nearest_valid_date(date, sp500_data):
    """Find the nearest valid date in the data."""
    # Try to find the exact date, otherwise check nearby dates
    if str(date.date()) in sp500_data.index:
        return str(date.date())
    else:
        # Check for the previous valid trading day
        for delta in range(1, 5):  # Look for up to 4 days back
            prev_date = date - pd.Timedelta(days=delta)
            if str(prev_date.date()) in sp500_data.index:
                return str(prev_date.date())
        # If no valid date, return None
        return None

def get_sp500_metrics(row):
    # Parse pubDate into a pandas datetime object and truncate time
    date = pd.to_datetime(row['pubDate']).normalize()  # Convert and normalize
    
    # Fetch S&P 500 historical data
    sp500_data = fetch_sp500_data(date)
    if sp500_data.empty:
        # If no data is returned, provide None values
        return pd.Series({
            'SP500_Open': None,
            'SP500_Close': None,
            'Day_Before_Close': None,
            'Two_Days_Later_Close': None,
            'Week_Later_Close': None,
        })

    try:
        # Attempt to get the valid date for each necessary point
        current_date = get_nearest_valid_date(date, sp500_data)
        day_before_date = get_nearest_valid_date(date - pd.Timedelta(days=1), sp500_data)
        two_days_later_date = get_nearest_valid_date(date + pd.Timedelta(days=2), sp500_data)
        week_later_date = get_nearest_valid_date(date + pd.Timedelta(days=7), sp500_data)

        return pd.Series({
            'SP500_Open': sp500_data.loc[current_date]['Open'] if current_date else None,
            'SP500_Close': sp500_data.loc[current_date]['Close'] if current_date else None,
            'Day_Before_Close': sp500_data.loc[day_before_date]['Close'] if day_before_date else None,
            'Two_Days_Later_Close': sp500_data.loc[two_days_later_date]['Close'] if two_days_later_date else None,
            'Week_Later_Close': sp500_data.loc[week_later_date]['Close'] if week_later_date else None,
        })
    except KeyError:  # Handle missing rows due to holidays or weekends
        return pd.Series({
            'SP500_Open': None,
            'SP500_Close': None,
            'Day_Before_Close': None,
            'Two_Days_Later_Close': None,
            'Week_Later_Close': None,
        })
    

df['pubDate'] = pd.to_datetime(df['pubDate'])  # Ensure pubDate is datetime
df_metrics = df.apply(get_sp500_metrics, axis=1)
df = pd.concat([df, df_metrics], axis=1)
print(df)


                                                  title             pubDate  \
0               recession now... or stagflation forever 2024-09-13 16:45:00   
1     mortgage rates are dropping, but homes are not... 2024-09-13 16:40:00   
2     hospitality stocks out-innovate challenges as ... 2024-09-13 16:34:47   
3     ratings agency fitch says extended strike at b... 2024-09-13 16:30:33   
4     google parent company in bear territory, down ... 2024-09-13 16:28:47   
...                                                 ...                 ...   
4632  a bull market is here: 2 brilliant stocks down... 2024-11-30 10:00:00   
4633  7 christmas movies you can watch with hulu's 9... 2024-11-30 10:00:00   
4634  jefferies: 'are we near the rebirth of new nuk... 2024-11-30 10:00:00   
4635  deutsche bank is out with its 2025 outlook for... 2024-11-30 10:00:00   
4636  mobile video game trailer, custom sports frami... 2024-11-30 10:00:00   

       SP500_Open  SP500_Close  Day_Before_Close  T

In [40]:
df['SP500_MA_3'] = df['SP500_Close'].rolling(window=3).mean()
df['SP500_MA_7'] = df['SP500_Close'].rolling(window=7).mean()

In [46]:
df2 = pd.read_csv('articles_with_sp500.csv')

df2.head()

,title,pubDate,SP500_Open,SP500_Close,Day_Before_Close,Two_Days_Later_Close,Week_Later_Close,SP500_MA_3,SP500_MA_7
0,recession now... or stagflation forever,2024-09-13 16:45:00,5603.339844,5626.02002,5595.759766,5626.02002,5713.640137,NaN,NaN
1,"mortgage rates are dropping, but homes are not...",2024-09-13 16:40:00,5603.339844,5626.02002,5595.759766,5626.02002,5713.640137,NaN,NaN
2,hospitality stocks out-innovate challenges as ...,2024-09-13 16:34:47,5603.339844,5626.02002,5595.759766,5626.02002,5713.640137,5626.02002,NaN
3,ratings agency fitch says extended strike at b...,2024-09-13 16:30:33,5603.339844,5626.02002,5595.759766,5626.02002,5713.640137,5626.02002,NaN
4,"google parent company in bear territory, down ...",2024-09-13 16:28:47,5603.339844,5626.02002,5595.759766,5626.02002,5713.640137,5626.02002,NaN
